In [406]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve,roc_auc_score

from tensorflow.keras.callbacks import EarlyStopping


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense,Dropout


from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows' , None)
pd.set_option('display.max_columns' , None)

In [407]:
test = pd.read_csv('test_l0Auv8Q.csv')
train = pd.read_csv('Train.csv')
first_health_camp = pd.read_csv('First_Health_Camp_Attended.csv')
second_health_camp = pd.read_csv('Second_Health_Camp_Attended.csv')
patient_profile = pd.read_csv('Patient_Profile.csv')
third_health_camp = pd.read_csv('Third_Health_Camp_Attended.csv')
health_camp = pd.read_csv('Health_Camp_Detail.csv')
sub=pd.read_csv('sample_submmission.csv')

In [408]:
train.dropna(inplace = True)

In [409]:
train.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [410]:
health_camp.head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [411]:
first_health_camp.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [412]:
second_health_camp.head()

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181
3,515398,6536,0.722041
4,504624,6536,0.464712


In [413]:
third_health_camp.head()

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [414]:
first_health_camp.drop(columns = ['Unnamed: 4'] , inplace = True)

In [415]:
def convert_datetime(x):
    
    return datetime.strptime(x ,'%d-%b-%y')

In [416]:
train['Registration_Date'] = train['Registration_Date'].apply(convert_datetime)
test['Registration_Date'] = test['Registration_Date'].apply(convert_datetime)

In [417]:
health_camp['Camp_Start_Date'] = health_camp['Camp_Start_Date'].apply(convert_datetime)

In [418]:
health_camp['Camp_End_Date'] = health_camp['Camp_End_Date'].apply(convert_datetime)

In [419]:
patient_profile['First_Interaction'] = patient_profile['First_Interaction'].apply(convert_datetime)

In [420]:
def change_to_int(r):

    if r == 'None':
        return None
    else:
        return float(r)

In [421]:
def Age_fill(x):
    if x=='None':
        return 30.0
    else:
        return int(x)
    
patient_profile['Age'] = patient_profile['Age'].apply(Age_fill)

In [422]:
patient_profile['Education_Score'] = patient_profile['Education_Score'].apply(change_to_int)

In [423]:
patient_profile['Education_Score'].min()

45.0

In [424]:
patient_profile['Education_Score'] = pd.to_numeric(patient_profile['Education_Score'] , errors = 'coerce').fillna(0).astype(np.float64)

In [425]:
patient_profile['Income'] = patient_profile['Income'].apply(change_to_int)

In [426]:
patient_profile['Employer_Category'].fillna('Unemployed' , inplace = True)

In [427]:
patient_profile['Income'] = patient_profile['Income'].apply(lambda x: 0 if pd.isnull(x) else x)

In [428]:
def fill_city(x):
    City = x[0]
    Income = x[1]
    if pd.isnull(City):
        if Income ==0:
            return 'H'
        else:
            return 'B'
        
    else:
        return City

In [429]:
patient_profile['City_Type'] = patient_profile[['City_Type' , 'Income']].apply(fill_city,axis=1)

In [430]:
df=pd.merge(train,health_camp,how='left',on=['Health_Camp_ID'])
df_test=pd.merge(test,health_camp,how='left',on=['Health_Camp_ID'])

In [431]:
df_1=pd.merge(df,patient_profile,how='left',on=['Patient_ID'])
df_test_final=pd.merge(df_test,patient_profile,how='left',on=['Patient_ID'])

In [432]:
df_2=pd.merge(df_1,first_health_camp,how='left',on=['Patient_ID','Health_Camp_ID'])

In [433]:
df_3=pd.merge(df_2,second_health_camp,how='left',on=['Patient_ID', 'Health_Camp_ID'])

In [434]:
df_final=pd.merge(df_3,third_health_camp,how='left',on=['Patient_ID', 'Health_Camp_ID'])

In [435]:
df_final['Category1'].value_counts()

First     49710
Second    14963
Third     10271
Name: Category1, dtype: int64

In [436]:
df_final.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,489652,6578,2005-09-10,4,0,0,0,2,2005-08-16,2005-10-14,Third,G,2,0,0,0,0,0.0,0.0,30.0,2004-12-06,H,Unemployed,NaN,NaN,NaN,2.0,1.0
1,507246,6578,2005-08-18,45,5,0,0,7,2005-08-16,2005-10-14,Third,G,2,0,0,0,0,1.0,75.0,40.0,2004-09-08,C,Others,NaN,NaN,NaN,NaN,NaN
2,523729,6534,2006-04-29,0,0,0,0,0,2005-10-17,2007-11-07,Second,A,2,0,0,0,0,0.0,0.0,30.0,2004-06-22,H,Unemployed,NaN,NaN,0.402054,NaN,NaN
3,524931,6535,2004-02-07,0,0,0,0,0,2004-02-01,2004-02-18,First,E,2,0,0,0,0,0.0,0.0,30.0,2004-02-07,I,Unemployed,NaN,NaN,NaN,NaN,NaN
4,521364,6529,2006-02-28,15,1,0,0,7,2006-03-30,2006-04-03,Second,A,2,0,0,0,1,1.0,70.0,40.0,2003-07-04,I,Technology,NaN,NaN,0.845597,NaN,NaN


In [437]:
df_final['Camp_Start_Date'].value_counts()

2005-09-27    10402
2005-02-19     4361
2004-01-09     4357
2005-06-13     4144
2006-03-30     3823
2005-01-03     3809
2005-10-17     3597
2005-07-09     3562
2004-12-22     3515
2005-08-16     2837
2004-10-01     2622
2004-11-24     2334
2005-06-19     2301
2005-02-23     2084
2005-10-09     2076
2004-08-07     1992
2005-02-15     1902
2004-02-01     1882
2005-06-23     1832
2006-02-10     1743
2005-09-15     1725
2005-12-03     1547
2003-12-07     1467
2004-11-01     1410
2003-11-22     1396
2003-12-17      514
2003-08-16      259
2003-11-30      200
2004-09-17      177
2004-09-09      171
2005-05-13      148
2003-11-03      128
2004-12-09      120
2004-04-26       94
2005-10-12       90
2004-05-21       82
2005-08-02       79
2004-04-29       66
2004-01-04       52
2004-08-22       44
Name: Camp_Start_Date, dtype: int64

In [438]:
# assert len(np.intersect1d(df_final['Camp_Start_Date'].unique(), df_test_final['Camp_Start_Date'].unique())) == len(df_test_final['Camp_Start_Date'].unique())

In [439]:
# df_final['Camp_Start_Date_Day']=df_final['Camp_Start_Date'].dt.day
df_final['Camp_Start_Date_Month']=df_final['Camp_Start_Date'].dt.month
df_final['Camp_Start_Date_Year']=df_final['Camp_Start_Date'].dt.year

# df_test_final['Camp_Start_Date_Day']=df_test_final['Camp_Start_Date'].dt.day
df_test_final['Camp_Start_Date_Month']=df_test_final['Camp_Start_Date'].dt.month
df_test_final['Camp_Start_Date_Year']=df_test_final['Camp_Start_Date'].dt.year

# df_final['Camp_End_Date_Day']=df_final['Camp_End_Date'].dt.day
# df_final['Camp_End_Date_Month']=df_final['Camp_End_Date'].dt.month
# df_final['Camp_End_Date_Year']=df_final['Camp_End_Date'].dt.year

# df_test_final['Camp_End_Date_Day']=df_test_final['Camp_End_Date'].dt.day
# df_test_final['Camp_End_Date_Month']=df_test_final['Camp_End_Date'].dt.month
# df_test_final['Camp_End_Date_Year']=df_test_final['Camp_End_Date'].dt.year

# df_final['Registration_Date_Day']=df_final['Registration_Date'].dt.day
# df_final['Registration_Date_Month']=df_final['Registration_Date'].dt.month
# df_final['Registration_Date_Year']=df_final['Registration_Date'].dt.year

# df_test_final['Registration_Date_Day']=df_test_final['Registration_Date'].dt.day
# df_test_final['Registration_Date_Month']=df_test_final['Registration_Date'].dt.month
# df_test_final['Registration_Date_Year']=df_test_final['Registration_Date'].dt.year

# df_final['First_Interaction_Day']=df_final['First_Interaction'].dt.day
# df_final['First_Interaction_Month']=df_final['First_Interaction'].dt.month
# df_final['First_Interaction_Year']=df_final['First_Interaction'].dt.year

# df_test_final['First_Interaction_Day']=df_test_final['First_Interaction'].dt.day
# df_test_final['First_Interaction_Month']=df_test_final['First_Interaction'].dt.month
# df_test_final['First_Interaction_Year']=df_test_final['First_Interaction'].dt.year

In [440]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74944 entries, 0 to 74943
Data columns (total 30 columns):
Patient_ID                   74944 non-null int64
Health_Camp_ID               74944 non-null int64
Registration_Date            74944 non-null datetime64[ns]
Var1                         74944 non-null int64
Var2                         74944 non-null int64
Var3                         74944 non-null int64
Var4                         74944 non-null int64
Var5                         74944 non-null int64
Camp_Start_Date              74944 non-null datetime64[ns]
Camp_End_Date                74944 non-null datetime64[ns]
Category1                    74944 non-null object
Category2                    74944 non-null object
Category3                    74944 non-null int64
Online_Follower              74944 non-null int64
LinkedIn_Shared              74944 non-null int64
Twitter_Shared               74944 non-null int64
Facebook_Shared              74944 non-null int64
Income      

In [441]:
# df_final.corr()['Outcomes']

In [442]:
def find_outcomes(x):
    first = x[0]
    second = x[1]
    third = x[2]
    
    if pd.isnull(first)==False:
        return 1
    elif pd.isnull(second)==False:
        return 1
    elif pd.isnull(third)==False:
        if third>0:
            return 1
        else:
            return 0
    else:
        return 0

In [443]:
df_final['Outcomes'] = df_final[['Health_Score','Health Score','Number_of_stall_visited']].apply(find_outcomes,axis=1)

In [444]:
len(df_final)

74944

In [445]:
df_final['Outcomes'].value_counts()

0    54606
1    20338
Name: Outcomes, dtype: int64

In [446]:
df_final.groupby('Outcomes')['City_Type'].value_counts()

Outcomes  City_Type
0         H            29730
          B             6181
          D             3741
          G             2912
          C             2859
          E             2732
          A             2354
          I             2229
          F             1868
1         H             8962
          B             2555
          D             1687
          G             1408
          C             1387
          E             1271
          A             1077
          I             1066
          F              925
Name: City_Type, dtype: int64

In [447]:
df_final.groupby('Outcomes')['Education_Score'].mean()

Outcomes
0     9.041519
1    13.593475
Name: Education_Score, dtype: float64

In [448]:
drop_cols=['Patient_ID','Health_Camp_ID','Registration_Date','Camp_Start_Date','Camp_End_Date','First_Interaction'
           ,'Donation','Health_Score','Health Score','Number_of_stall_visited','Last_Stall_Visited_Number'
           ,'Education_Score']
drop_cols_test=['Patient_ID','Health_Camp_ID','Registration_Date','Camp_Start_Date','Camp_End_Date',
                'First_Interaction','Education_Score']

In [449]:
train_final=df_final.drop(columns=drop_cols)
test_final=df_test_final.drop(columns=drop_cols_test)

In [450]:
train_final.corr()['Outcomes']

Var1                     0.045339
Var2                     0.033361
Var3                     0.014598
Var4                     0.032495
Var5                     0.064405
Category3                0.002634
Online_Follower          0.048703
LinkedIn_Shared          0.053411
Twitter_Shared           0.044255
Facebook_Shared          0.038038
Income                   0.061721
Age                      0.077350
Camp_Start_Date_Month   -0.045535
Camp_Start_Date_Year     0.225239
Outcomes                 1.000000
Name: Outcomes, dtype: float64

In [451]:
# label_cols = ['Category1']
dummie_cols=['Employer_Category','Category2','City_Type','Income','Category1']

In [452]:
# for feature in label_cols:
#     labels_ordered=train_final.groupby([feature])['Outcomes'].mean().sort_values().index
#     labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
#     train_final[feature]=train_final[feature].map(labels_ordered)
#     test_final[feature]=test_final[feature].map(labels_ordered)

In [453]:
train_final=pd.get_dummies(train_final,columns=dummie_cols,drop_first=True)
test_final=pd.get_dummies(test_final,columns=dummie_cols,drop_first=True)

In [454]:
train_final.corr()['Outcomes']

Var1                                   0.045339
Var2                                   0.033361
Var3                                   0.014598
Var4                                   0.032495
Var5                                   0.064405
Category3                              0.002634
Online_Follower                        0.048703
LinkedIn_Shared                        0.053411
Twitter_Shared                         0.044255
Facebook_Shared                        0.038038
Age                                    0.077350
Camp_Start_Date_Month                 -0.045535
Camp_Start_Date_Year                   0.225239
Outcomes                               1.000000
Employer_Category_Broadcasting        -0.001779
Employer_Category_Consulting           0.020915
Employer_Category_Education            0.024316
Employer_Category_Food                 0.002102
Employer_Category_Health               0.006682
Employer_Category_Manufacturing        0.001653
Employer_Category_Others               0

In [455]:
X=train_final.drop('Outcomes',axis=1)
y=train_final['Outcomes']

In [456]:
train_final.shape

(74944, 50)

In [457]:
train_final.head()

,Var1,Var2,Var3,Var4,Var5,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Age,Camp_Start_Date_Month,Camp_Start_Date_Year,Outcomes,Employer_Category_Broadcasting,Employer_Category_Consulting,Employer_Category_Education,Employer_Category_Food,Employer_Category_Health,Employer_Category_Manufacturing,Employer_Category_Others,Employer_Category_Real Estate,Employer_Category_Retail,Employer_Category_Software Industry,Employer_Category_Technology,Employer_Category_Telecom,Employer_Category_Transport,Employer_Category_Unemployed,Category2_B,Category2_C,Category2_D,Category2_E,Category2_F,Category2_G,City_Type_B,City_Type_C,City_Type_D,City_Type_E,City_Type_F,City_Type_G,City_Type_H,City_Type_I,Income_1.0,Income_2.0,Income_3.0,Income_4.0,Income_5.0,Income_6.0,Category1_Second,Category1_Third
0,4,0,0,0,2,2,0,0,0,0,30.0,8,2005,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,45,5,0,0,7,2,0,0,0,0,40.0,8,2005,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,2,0,0,0,0,30.0,10,2005,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,2,0,0,0,0,30.0,2,2004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,15,1,0,0,7,2,0,0,0,1,40.0,3,2006,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0


In [458]:
test_final.shape

(35249, 48)

In [459]:
test_final.head()

,Var1,Var2,Var3,Var4,Var5,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Age,Camp_Start_Date_Month,Camp_Start_Date_Year,Employer_Category_Broadcasting,Employer_Category_Consulting,Employer_Category_Education,Employer_Category_Food,Employer_Category_Health,Employer_Category_Manufacturing,Employer_Category_Others,Employer_Category_Real Estate,Employer_Category_Retail,Employer_Category_Software Industry,Employer_Category_Technology,Employer_Category_Telecom,Employer_Category_Transport,Employer_Category_Unemployed,Category2_C,Category2_D,Category2_E,Category2_F,Category2_G,City_Type_B,City_Type_C,City_Type_D,City_Type_E,City_Type_F,City_Type_G,City_Type_H,City_Type_I,Income_1.0,Income_2.0,Income_3.0,Income_4.0,Income_5.0,Income_6.0,Category1_Second,Category1_Third
0,1,0,0,0,2,2,0,0,0,0,44.0,6,2006,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,2,0,1,0,0,41.0,8,2006,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
2,0,0,0,0,0,2,0,0,0,0,30.0,4,2006,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,2,0,0,0,0,47.0,11,2006,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,2,0,0,0,0,30.0,9,2006,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [460]:
t=train_final.drop('Outcomes',axis=1).copy()

In [461]:
missing_cols = set( t ) - set( test_final.columns )

In [462]:
missing_cols

{'Category2_B'}

In [463]:
missing_cols = set( t ) - set( test_final.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    test_final[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
test_final = test_final[t.columns]

In [464]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=101)

In [465]:
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
test_final=scaler.transform(test_final)

In [466]:
model_lr=LogisticRegression()
model_lr.fit(X_train,y_train)
pred_lr=model_lr.predict_proba(X_test)[:,1]

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [467]:
pred_test_lr=model_lr.predict_proba(test_final)[:,1]

In [468]:
lr_auc=roc_auc_score(y_test,pred_lr)
lr_auc

0.8097344548736299

In [469]:
sub['Outcome']=pred_test_lr
sub.to_csv('LogReg_HealthCare.csv',index=False)
sub.head()

,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.727792
1,500633,6584,0.661574
2,506945,6582,0.101057
3,497447,6551,0.634058
4,496446,6533,0.091877


In [470]:
model_rf = RandomForestClassifier(random_state=101)
model_rf.fit(X_train,y_train)
pred_rf=model_rf.predict_proba(X_test)[:,1]

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [471]:
pred_rf.shape

(14989,)

In [472]:
rf_auc=roc_auc_score(y_test,pred_rf)
rf_auc

0.8157056226538548

In [473]:
pred_test_rf=model_rf.predict_proba(test_final)[:,1]

In [474]:
sub['Outcome']=pred_test_rf
sub.to_csv('RF_HealthCare.csv',index=False)
sub.head()

,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.866667
1,500633,6584,0.300000
2,506945,6582,0.162903
3,497447,6551,0.506667
4,496446,6533,0.133818


In [475]:
model_xgb =XGBClassifier()
model_xgb.fit(X_train,y_train)
pred_xgb=model_xgb.predict_proba(X_test)[:,1]

In [476]:
xgb_auc=roc_auc_score(y_test,pred_xgb)
xgb_auc

0.8381785088325794

In [477]:
pred_test_xgb=model_xgb.predict_proba(test_final)[:,1]

In [478]:
sub['Outcome']=pred_test_xgb
sub.to_csv('XGB_HealthCare_19.csv',index=False)
sub.head()

,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.636751
1,500633,6584,0.462328
2,506945,6582,0.074090
3,497447,6551,0.640402
4,496446,6533,0.049652


In [479]:
X_train.shape

(59955, 49)

In [480]:
# model=Sequential()

# model.add(Dense(33,input_shape=(33,),activation='relu'))

# model.add(Dense(64,activation='relu'))

# model.add(Dense(128,activation='relu'))

# model.add(Dense(256,activation='relu'))

# model.add(Dense(1,activation='sigmoid'))

# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [481]:
# early_stop=EarlyStopping(monitor='val_accuracy',mode='min',verbose=1,patience=25)

In [482]:
# y_train1=np.array(y_train)
# y_test1=np.array(y_test)

In [483]:
# model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=[early_stop])

In [484]:
# pred_learning=model.predict_proba(X_test)

In [485]:
# pred_test_learning=model.predict_proba(test_final)

In [486]:
# learning_auc=roc_auc_score(y_test,pred_learning)
# learning_auc

In [487]:
# pred_test_learning

In [488]:
# sub['Outcome']=pred_test_learning
# sub.to_csv('Deep_HealthCare_5.csv',index=False)
# sub.head()